In [2]:
# LIBRARYs

import torch
import torchvision
import torchvision.transforms as transforms
import torch.nn as nn
import torch.nn.functional as F
import torch.backends.cudnn as cudnn
import torch.optim as optim
import os
import argparse

import wandb

from Utils.treino import *
from Utils.init_redes import *
from Utils.redes import *
from Utils.data_set import *

cuda


In [3]:
if torch.cuda.is_available():
    device = 'cuda'  
else:
    device = 'cpu'
print(device)

best_acc = 0  # best test accuracy
start_epoch = 0  # start from epoch 0 or last checkpoint epoch

cuda


In [4]:
# DATASET
data_set = "Cifar100"

if data_set == "Cifar10":
    transform_train, transform_test, batch_size, trainset, trainloader, testset, testloader, classes = cifar10()
elif data_set == "Cifar100":
    transform_train, transform_test, batch_size, trainset, trainloader, testset, testloader= cifar100()

Files already downloaded and verified
Files already downloaded and verified


In [ ]:
num_epoch = 120

data_sets = ["Cifar100", "Cifar10"]
for data_set in data_sets:
    if data_set == "Cifar10":
        transform_train, transform_test, batch_size, trainset, trainloader, testset, testloader, classes = cifar10()
        rede = "Resnet_hydra"
        max_ = 16
    elif data_set == "Cifar100":
        transform_train, transform_test, batch_size, trainset, trainloader, testset, testloader= cifar100()
        rede = "Resnet100_hydra"
        max_ = 15
 
    for i in range(2,max_):
        num_cabeca = i
        net, loss_criterion, optimizer_net, scheduler_net = init_rede_all(rede, num_cabeca)

        name = "Hydra_{}_cabecas".format(num_cabeca)

        config = dict (
          architecture = "Resnet18 Hydra",
          N_cabeças = num_cabeca,
          Data_set = data_set,
          epocas = num_epoch,
        )

        wandb.init(project="Hydra", config=config, name="{} Cabeças".format(num_cabeca), group=data_set)

        best_acc = 0
        best_loss = 100
        for epoch in range(start_epoch, num_epoch):
            print(epoch, end=" ")
            train_loss, train_acc = train(epoch=epoch, model=net, loss_criterion=loss_criterion, optimizer=optimizer_net, scheduler=scheduler_net,trainloader=trainloader)
            test_loss, test_acc, best_acc, best_loss = test(epoch=epoch, model=net, loss_criterion=loss_criterion, optimizer=optimizer_net, scheduler=scheduler_net, best_acc=best_acc, best_loss=best_loss,name=name,testloader=testloader)
            print("Train acc: {} | Train Loss: {} || Teste acc: {} | Teste Loss: {}".format(train_acc, train_loss, test_acc, test_loss))
            wandb.log({"epoch": epoch, "Loss Treino": train_loss, "acuracia treino": train_acc, "acuracia teste": test_acc, "Loss Teste": test_loss})

        wandb.finish()

        hydra, _,__,___  = init_rede_all(rede, num_cabeca)
        hydra.load_state_dict(torch.load("./Resultado_parciais/{}_acc.pth".format(name)))

        acc_teste = testa_acuracia(hydra, testloader)
        torch.save(hydra.state_dict(), ('./Data_sets/Hydra {}/Hydra_{}_cabecas_acc_{}.pth'.format(data_set,num_cabeca,acc_teste)))

Files already downloaded and verified
Files already downloaded and verified


wandb: Currently logged in as: luigidoria. Use `wandb login --relogin` to force relogin


0 Train acc: 8.316 | Train Loss: 1556.262865304947 || Teste acc: 12.51 | Teste Loss: 369.599609375
1 